In [1]:
user_input = input('Input the comedy transcript you would like to predict: ')
user_input

'Hey, how\'s it going, folks? I recently went on a blind date, and it was an experience. The guy showed up wearing a fedora and carrying a cane, which was odd because he wasn\'t blind. I think he was going for a 1920s gangster look or something. But I was willing to give him a chance.  So, we went to a restaurant, and he ordered for me, which was sweet. But then he ordered the most expensive thing on the menu, and I was like, "Dude, I don\'t know you like that." I mean, I appreciate a fancy dinner, but I\'m not trying to go into debt on a first date.  Then he started telling me about his job, and it turned out he was a professional mime. I didn\'t even know that was a thing. I asked him how he got into it, and he just mimed a ladder and climbed it. So, I guess he\'s always been a bit of a clown.  The conversation was getting a bit awkward, so I decided to break the tension with a joke. I said, "Why did the mime cross the road?" And he just stood there, staring at me. And I was like, "T

In [2]:
import pandas as pd
from pipeline_functions import *
from imblearn.over_sampling import SMOTE
import pickle

In [3]:
valid_transcripts_sent_df = pd.read_pickle('chloe_valid_transcripts_sent_df')
valid_transcripts_sent_df

,Comedian,Title,Subtitle,Transcript Part,Action,Processed Transcript,Audience Reaction,Funniness
0,Chris Rock,Selective Outrage (2023) | Transcript,NaN,Let’s go!,[audience cheering],let go,cheer,1
1,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"She said, “$300, I’ll do anything you want.” ...",[audience laughing] [audience cheering],said anything want said bitch paint house ...,cheer laugh,2
2,Chris Rock,Selective Outrage (2023) | Transcript,NaN,Ladies and gentlemen. Ladies and gentlemen. C...,[audience cheering] [audience continue cheering],lady gentleman lady gentleman chris rock,cheer,1
3,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"What’s up, Baltimore?",[audience cheers loudly],baltimore,cheer,1
4,Chris Rock,Selective Outrage (2023) | Transcript,NaN,"Yes! Yes, yes. Thank you! Thank you so much! ...",[audience cheering],yes yes yes thank thank much thank much coming...,cheer,1
...,...,...,...,...,...,...,...,...
6487,NaN,Jim Jefferies on Gun Control [Full Transcript],Australian comedian Jim Jefferies breaks down ...,"“Guns! Who wants to sell me a gun?” Now, I’m ...",[Audience cheering],gun want sell gun gon na wrap talk anymore...,cheer,1
6488,NaN,Jim Jefferies on Gun Control [Full Transcript],Australian comedian Jim Jefferies breaks down ...,Yeah! And that made a hell of a lot of sense ...,NA,yeah made hell lot sense musket know governmen...,NA,0
6489,Reggie Watts,Spatial (2016) – Full Transcript,Completely improvised show weaving together sk...,” Fuck. # ..Then jump for my love # Jump in # ...,NA,fuck jump love jump feel touch jump wan na ta...,NA,0
6490,GEORGE CARLIN,COMPLAINTS AND GRIEVANCES (2001) – FULL TRANSC...,"Full transcript of Complaints and Grievances, ...",” Now. Folks. This next piece of material’s go...,NA,folk next piece material going give u chance...,NA,0


In [4]:
valid_transcripts_df = pd.read_pickle('valid_transcripts_df')
valid_transcripts_df

,Unnamed: 0,Comedian,Date,Title,Subtitle,Transcript,Cleaned Transcript,Polarity,Subjectivity
0,0,Chris Rock,"March 8, 2023",Selective Outrage (2023) | Transcript,NaN,[slow instrumental music playing] [funk drums ...,lets go she said ill do anything you w...,0.053924,0.537392
1,1,Marc Maron,"March 3, 2023",Thinky Pain (2013) | Transcript,Marc Maron returns to his old stomping grounds...,[siren wailing] I don’t know what you were thi...,i dont know what you were thinking like im no...,0.039222,0.527340
6,6,Dave Chappelle,"January 22, 2023",What’s in a Name (2022) | Transcript,Dave Chappelle delivers a speech at his presti...,What’s in a Name? is a 40-minute talk Chappell...,whats in a name is a talk chappelle delivered...,0.131587,0.528296
7,7,Iliza Shlesinger,"December 20, 2022",Hot Forever (2022) | Transcript,With topics ranging from tight rompers to ugly...,[upbeat music playing] [crowd cheering] Clevel...,cleveland ohio thank you thank you so much t...,0.093386,0.510377
8,8,Gabriel Iglesias,"November 28, 2022",Stadium Fluffy (2022) | Transcript,"Features Gabriel ""Fluffy"" Iglesias as he talks...",[man] Can you please state your name? Martin M...,can you please state your name martin moreno ...,0.110201,0.490649
...,...,...,...,...,...,...,...,...,...
410,410,George Carlin,"April 6, 2017",Jamming in New York (1992) – Transcript,NaN,Jammin’ in New York is George Carlin’s 14th al...,jammin in new york is george carlins album an...,0.031729,0.525063
411,411,NaN,"April 1, 2017",Jim Jefferies on Gun Control [Full Transcript],Australian comedian Jim Jefferies breaks down ...,Australian comedian Jim Jefferies breaks down ...,australian comedian jim jefferies breaks down ...,-0.000192,0.537790
412,412,Reggie Watts,"March 30, 2017",Spatial (2016) – Full Transcript,Completely improvised show weaving together sk...,"Hello, I’m Thomas. I’m so glad to meet you Mum...",hello im thomas im so glad to meet you mum im...,0.104515,0.569039
413,413,GEORGE CARLIN,"March 29, 2017",COMPLAINTS AND GRIEVANCES (2001) – FULL TRANSC...,"Full transcript of Complaints and Grievances, ...",Complaints and Grievances is a HBO stand-up sp...,complaints and grievances is a hbo standup spe...,0.025769,0.506590


In [5]:
#filter transcripts in 'valid_transcripts_df' for those similar to 'user_input' in terms of polarity and subjectivity
filtered_transcripts_df = filter_by_sentiment(valid_transcripts_df, user_input)
filtered_transcripts_df

,Unnamed: 0,Comedian,Date,Title,Subtitle,Transcript,Cleaned Transcript,Polarity,Subjectivity
0,143,Eddie Griffin,"July 24, 2019",The Black Man’s Bible (1994) – Transcript,NaN,HBO Comedy Half-Hour Episode aired 21 July 199...,hbo comedy halfhour episode aired july they ...,-0.101926,0.534573
1,43,Dave Chappelle,"August 5, 2021",8:46 – Transcript,Chappelle touches on Floyd's death and subsequ...,"June 6, 2020 8:46 is a performance special by ...",june is a performance special by comedian d...,-0.041358,0.501230
2,74,GEORGE LOPEZ,"July 14, 2020",WE’LL DO IT FOR HALF (2020) – FULL TRANSCRIPT,Lopez spends the majority of his time onstage ...,"San Francisco, it’s my honor to introduce to y...",san francisco its my honor to introduce to you...,-0.085255,0.579037
3,329,John Mulaney,"September 25, 2017",New In Town (2012) – Transcript,Stand-up comedian John Mulaney tackles such re...,[funky 90’s beat and cityscape pan] [singing] ...,♬ new in town ♬ john mulaneys new in town ...,0.049306,0.501172
4,260,D.L. Hughley,"January 19, 2018",Unapologetic (2007) – Transcript,"D.C., Hughley focuses on such topics as the da...",[Audience cheering] (DL Hughley enters from st...,dl hughley enters from stage left with his he...,-0.079306,0.513705
5,181,Ellen Degeneres,"December 22, 2018",Relatable (2018) – Transcript,"In her first special since 2003, Ellen revisit...",A NETFLIX ORIGINAL COMEDY SPECIAL [crowd cheer...,a netflix original comedy special thank you ...,0.121846,0.501824
6,149,Jo Koy,"June 16, 2019",Live from Seattle (2017) – Full Transcript,"Seventh grade math can be tough, even for grow...",Yo Finally here You know I’m finally here Fina...,yo finally here you know im finally here final...,-0.069408,0.594460
7,365,JIM JEFFERIES,"July 27, 2017",CONTRABAND (2008) – Full Transcript,NaN,"Ladies and Gentlemen, welcome to the Art theat...",ladies and gentlemen welcome to the art theatr...,0.039505,0.501962
8,221,Bill Burr,"July 6, 2018",The Philadelphia Incident (2006) – Transcript,"Transcript of the ""Philadelphia incident"" wher...",NEW! The full transcript of Bill’s monologue ...,new the full transcript of bills monologue at...,-0.144470,0.523386
9,246,NaN,"February 22, 2018",Eddie Izzard – Glorious (1997) – Transcript,Eddie Izzard's routine has a loose trajectory ...,"(Siren) Well, Times Square. I just had to do a...",siren well times square i just had to do a sho...,0.015845,0.500302


In [6]:
#find the rows in 'valid_transcripts_sent_df' whose title is in 'filtered_transcripts_df' 
filtered_transcripts_sent_df = valid_transcripts_sent_df[valid_transcripts_sent_df['Title'].isin(filtered_transcripts_df['Title'])]
filtered_transcripts_sent_df

,Comedian,Title,Subtitle,Transcript Part,Action,Processed Transcript,Audience Reaction,Funniness
1526,Dave Chappelle,8:46 – Transcript,Chappelle touches on Floyd's death and subsequ...,♫ Nobody Speak by DJ Shadow feat. Run the Jew...,NA,♫ nobody speak dj shadow feat run jewel ♫,NA,0
2889,GEORGE LOPEZ,WE’LL DO IT FOR HALF (2020) – FULL TRANSCRIPT,Lopez spends the majority of his time onstage ...,"San Francisco, it’s my honor to introduce to y...",[audience cheering],san francisco honor introduce voice generatio...,cheer,1
2890,GEORGE LOPEZ,WE’LL DO IT FOR HALF (2020) – FULL TRANSCRIPT,Lopez spends the majority of his time onstage ...,"I said, qué chingados!",[audience cheering],said qué chingados,cheer,1
2891,GEORGE LOPEZ,WE’LL DO IT FOR HALF (2020) – FULL TRANSCRIPT,Lopez spends the majority of his time onstage ...,For those of you who don’t understand… Same s...,[audience cheering and applauding],understand… shit let tell little bit name geo...,cheer applaud,1
2892,GEORGE LOPEZ,WE’LL DO IT FOR HALF (2020) – FULL TRANSCRIPT,Lopez spends the majority of his time onstage ...,I have one kidney that’s not mine… and dement...,[audience cheering and applauding],one kidney mine… dementia run family come fu...,cheer applaud,1
...,...,...,...,...,...,...,...,...
5953,John Mulaney,New In Town (2012) – Transcript,Stand-up comedian John Mulaney tackles such re...,". I don’t know why, I think its cause this has...",(Audience laughs),know think cause little remaining dignity kno...,laugh,2
5954,John Mulaney,New In Town (2012) – Transcript,Stand-up comedian John Mulaney tackles such re...,"This is sort of like, go stick it in, I am an...",(Audience laughs and claps),sort like go stick american like leaning edge...,laugh,2
5955,John Mulaney,New In Town (2012) – Transcript,Stand-up comedian John Mulaney tackles such re...,"But I didn’t say it, like it came from my voc...",(Audience laughs),say like came vocal chord totally involuntary...,laugh,2
5956,John Mulaney,New In Town (2012) – Transcript,Stand-up comedian John Mulaney tackles such re...,"So, he pulls his hand out and I thought I was...",(Audience laughs),pull hand thought shitting hand yelled sorry...,laugh,2


In [7]:
#TF-IDF vectorization on 'filtered_transcripts_sent_df' 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
filtered_tfidf_matrix = vectorizer.fit_transform(filtered_transcripts_sent_df['Processed Transcript'])

X_train_filtered = pd.DataFrame(filtered_tfidf_matrix.toarray(), columns = vectorizer.get_feature_names_out())
y_train_filtered = filtered_transcripts_sent_df['Funniness']

In [8]:
#get counts of each funniness in 'y_train_filtered'
y_train_filtered.value_counts()

2    334
1     49
0      9
3      3
Name: Funniness, dtype: int64

In [9]:
#oversample to account for inbalanced data
smote = SMOTE(k_neighbors = 1)
X_train_sm, y_train_sm = smote.fit_resample(X_train_filtered, y_train_filtered)
y_train_sm.value_counts()

0    334
1    334
2    334
3    334
Name: Funniness, dtype: int64

In [10]:
#train gradient boosting classifier model on X_train_sm, y_train_sm
from sklearn.ensemble import GradientBoostingClassifier

filtered_grad_clf = GradientBoostingClassifier(n_estimators = 80, learning_rate = 0.01, random_state = 0).fit(X_train_sm, y_train_sm)

In [11]:
#preprocess 'user_input' and use the vectorizer to transform it
processed_user_input = preprocess(user_input)
vec_user_input = vectorizer.transform([processed_user_input])
user_input_df = pd.DataFrame(vec_user_input.toarray(), columns = vectorizer.get_feature_names_out())

#make prediction of funniness of 'user_input' 
filtered_pred = filtered_grad_clf.predict(user_input_df)[0]

In [12]:
#get model that is pre-trained on all valid transcripts, and also the vectorizer, and features
full_grad_clf = pickle.load(open('trained_grad_model', 'rb'))
full_vectorizer = pickle.load(open('full_vectorizer', 'rb'))
full_features = pickle.load(open('full_features', 'rb'))

#transform processed 'user_input' using full_vectorizer, select the best 10000 features, and make prediction of funniness using full_grad_clf
fullvec_user_input = full_vectorizer.transform([processed_user_input])
full_user_input_df = pd.DataFrame(fullvec_user_input.toarray(), columns = full_vectorizer.get_feature_names_out())
full_user_input_df = full_user_input_df[full_features]
full_pred = full_grad_clf.predict(full_user_input_df)[0]

In [13]:
#function to output funniness prediction in word form
def get_pred_output(pred):
    if pred == 0:
        return 'Neutral'
    elif pred == 1:
        return 'A little funny'
    elif pred == 2:
        return 'Moderately funny'
    elif pred == 3:
        return 'Very funny'
    
print('Funniness based on similar transcripts: ', get_pred_output(filtered_pred))
print('Funniness based on all transcripts available: ', get_pred_output(full_pred))

Funniness based on similar transcripts:  Moderately funny
Funniness based on all transcripts available:  Moderately funny
